In [58]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True
%matplotlib notebook


DEBUG:matplotlib.pyplot:Loaded backend nbAgg version unknown.


In [59]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from config import SmartFactoryConfig
from scene_setup import setup_scene
from channel_generator import SmartFactoryChannel

# Create config and scene
config = SmartFactoryConfig()
scene = setup_scene(config)

# Create channel generator
channel_gen = SmartFactoryChannel(config, scene)

# Generate channel data
channel_data = channel_gen.generate_channel()

INFO:scene_setup:Starting scene setup with ray tracing...
DEBUG:scene_setup:Debugging Base station state:
DEBUG:scene_setup:- Position: [10.  10.   4.5]
DEBUG:scene_setup:- Orientation: [  0.   0. -90.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000002674000D350>
DEBUG:scene_setup:Debugging RIS state:
DEBUG:scene_setup:- Position: [10.  19.5  2.5]
DEBUG:scene_setup:- Orientation: [  0. -90.   0.]
DEBUG:scene_setup:Debugging AGV_0 state:
DEBUG:scene_setup:- Position: [12.   5.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x000002673BBC0250>
DEBUG:scene_setup:Debugging AGV_1 state:
DEBUG:scene_setup:- Position: [ 8.  15.   0.5]
DEBUG:scene_setup:- Orientation: [0. 0. 0.]
DEBUG:scene_setup:- Array config: <sionna.rt.antenna_array.PlanarArray object at 0x00000266CAA14690>
INFO:scene_setup:Scene setup completed successfully
INFO:scene_setup:- Transmitters: 7
INFO:sc

XLA can lead to reduced numerical precision. Use with care.
